In [ ]:
import json
import pickle
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv('avg_stats.csv')

In [ ]:
rosters = []
START_YEAR = 1980
for YEAR in range(START_YEAR,2020):
    rosters.append(json.load(open('rosters/auto_roster_{}.json'.format(YEAR))))


In [ ]:

y_map = { 'hgt': 'hgt',
   'stre': 'str',
   'spd': 'spd',
   'jmp': 'jmp',
   'endu': 'end',
   'ins': 'ins',
   'dnk': 'dnk',
   'ft': 'ft.1',
   'fg': '2pt',
   'tp': '3pt',
   'diq': 'diq',
   'oiq': 'oiq',
   'drb': 'drb',
   'pss': 'pss',
   'reb': 'reb' }
avgs = {}
for k in y_map:
    avgs[k + 'c'] = np.zeros(100)
    avgs[k + 'n'] = np.zeros(100)
    avgs[k + 's'] = np.zeros(100)
    avgs[k + 'sn'] = np.zeros(100)


In [ ]:
CURRENT_YEAR = 2019
from collections import defaultdict
player_years = defaultdict(list)
player_age_full = defaultdict(dict) # player 3

nc2 = np.array([5,1,4,2,1,1,2,1,1,3,7,3,3,3,1])
nc2 = nc2 /nc2.sum()


for i,roster in enumerate(rosters):
    for player in roster['players']:
        if player['tid'] <= -1:
            continue
        name = player['firstName'] + ' ' + player['lastName']

        vals = []
        age = (START_YEAR+i)-player['born']['year']
        for k,km in y_map.items():
            avgs[k + 'c'][age] += player['ratings'][0][k]
            avgs[k + 'n'][age] += 1#player['ratings'][0][k]
            vals.append(player['ratings'][0][k])
        player_age_full[name][age] = np.array(vals)

        #ovr = sum(player['ratings'][0][k]*nc2[y_keys.index(km)] for k,km in y_map.items())
        #if player['tid'] != -1:
        #    player_years[name].append((ovr,i+START_YEAR,[_ for _ in roster['teams'] if _['tid'] == player['tid']],player))

In [ ]:
for i,k in enumerate(y_map):
    avgs[k + 'DN'] = {}

for name, year_d in player_age_full.items():
    start,stop = min(year_d),max(year_d)
    for age in range(start,stop-1):
        for i,k in enumerate(y_map):
            try:
                avgs[k + 'DN'][age] = avgs[k + 'DN'].get(age,[]) + [year_d[age+1][i]-year_d[age][i]]
            except:
                pass
player_age_full['LeBron James']

In [ ]:
for k in y_map:
    avgs[k + 'm'] = avgs[k + 'c']/np.maximum(1,avgs[k + 'n'])

In [ ]:
for i,roster in enumerate(rosters):
    for player in roster['players']:
        if player['tid'] <= -1:
            continue
        name = player['firstName'] + ' ' + player['lastName']

        age = (START_YEAR+i)-player['born']['year']
        for k,km in y_map.items():
            avgs[k + 's'][age] += (player['ratings'][0][k]-avgs[k + 'm'][age])**2
            avgs[k + 'sn'][age] += 1#player['ratings'][0][k]

In [ ]:
for k in y_map:
    avgs[k + 'ss'] = np.sqrt(avgs[k + 's']/np.maximum(1,avgs[k + 'sn']))
    #print(k,avgs[k + 'ss'])

In [ ]:
bbm = np.zeros((100,15))
bbs = np.zeros((100,15))

for a in range(100):
    try:
        bbm[a] = df[df.Age==a].mean()[-15:]
        bbs[a] = df[df.Age==a].std()[-15:]
    except Exception as e:
        print(a,e)

In [ ]:
yidx = list(y_map.keys())

In [ ]:
plt.figure(figsize=(10,10))
for i, k in enumerate(y_map):
    plt.subplot(5,3,i+1)
    plt.plot(avgs[k + 'm'],lw=3,label='nba auto')
    plt.plot(bbm[:,yidx.index(k)],lw=3,label='bbgm')

    V = 0.0
    #plt.fill_between(np.arange(100),avgs[k + 'm']-V*avgs[k + 'ss'],avgs[k + 'm']+V*avgs[k + 'ss'],alpha=0.3)
    ms = (avgs[k + 'm']-V*avgs[k + 'ss'])
    ms = min(bbm[25:40,yidx.index(k)].min(),ms[np.where(ms > 0)].min())
    mxs = (avgs[k + 'm']+V*avgs[k + 'ss'])
    mxs = max(bbm[25:40,yidx.index(k)].max(),mxs.max())
    plt.title(k)
    plt.legend()
    plt.xlim(19,40)
    plt.ylim(ms,mxs)
plt.tight_layout()
plt.savefig('dev-comp.png',facecolor='w',edgecolor='w')

In [ ]:
for i,k in enumerate(y_map):
    avgs[k + 'DB'] = {}

for pid in df.pid.unique():
    mdf = df[df.pid == pid]
    v = np.array(df[df.pid == pid].sort_values('Age'))[:,-15:]
    v2 = np.diff(v,axis=0)
    start,stop = mdf.Age.min(), mdf.Age.max()
    for age in range(start,stop-1):
        for i,k in enumerate(y_map):
            try:
                avgs[k + 'DB'][age] = avgs[k + 'DB'].get(age,[]) + [v2[age-start][i]]
            except:
                pass
    #print(name)
for k in y_map:
    avgs[k + 'DBm'] = np.zeros(100)
    avgs[k + 'DBs'] = np.zeros(100)

    for age in range(100):
        if age in avgs[k + 'DB']:
            avgs[k + 'DBm'][age] = np.array(avgs[k + 'DB'][age]).mean()
            avgs[k + 'DBs'][age] = np.array(avgs[k + 'DB'][age]).std()

In [ ]:
for k in y_map:
    avgs[k + 'DNm'] = np.zeros(100)
    avgs[k + 'DNs'] = np.zeros(100)

    for age in range(100):
        if age in avgs[k + 'DN']:
            avgs[k + 'DNm'][age] = np.array(avgs[k + 'DN'][age]).mean()
            avgs[k + 'DNs'][age] = np.array(avgs[k + 'DN'][age]).std()

In [ ]:
plt.figure(figsize=(10,10))
for i, k in enumerate(y_map):
    plt.subplot(5,3,i+1)
    plt.plot(avgs[k + 'DNm'],lw=2,label='nba auto')
    plt.plot(avgs[k + 'DBm'],lw=2,label='BBGM')

    #plt.plot(bbm[:,yidx.index(k)],lw=3,label='bbgm')

    V = 0.5
    plt.fill_between(np.arange(100),avgs[k + 'DNm']-V*avgs[k + 'DNs'],avgs[k + 'DNm']+V*avgs[k + 'DNs'],alpha=0.3)
    plt.fill_between(np.arange(100),avgs[k + 'DBm']-V*avgs[k + 'DBs'],avgs[k + 'DBm']+V*avgs[k + 'DBs'],alpha=0.3)

    ms = (avgs[k + 'm']-V*avgs[k + 'ss'])
    ms = min(bbm[25:40,yidx.index(k)].min(),ms[np.where(ms > 0)].min())
    mxs = (avgs[k + 'm']+V*avgs[k + 'ss'])
    mxs = max(bbm[25:40,yidx.index(k)].max(),mxs.max())
    plt.title(k)
    plt.legend()
    plt.plot([0,100],[0,0],c='k',lw=2)
    plt.xlim(18,36)
    #plt.ylim(ms,mxs)
plt.tight_layout()
plt.savefig('dev-comp-AI4.png',facecolor='w',edgecolor='w')